[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JorisRoels/deep-learning-biology/blob/main/exercises/assignments/2020-dlb-6-generative-neural-networks.ipynb)

# Exercise 6: Generative neural networks

In this notebook, we will train a generative adversarial network to generate images of digits. Next, we will adapt this model to generate electron microscopy data.  

The structure of these exercises is as follows: 

1. [Import libraries and download data](#scrollTo=ScagUEMTMjlK)
2. [Designing a generative adversarial network](#scrollTo=KdbhNqzDZ2wm)
3. [Training a generative adversarial network](#scrollTo=ffGI9IoAyYzt)
4. [Adaptation to electron microscopy data](#scrollTo=ohZHyOTnI35b)

## 1. Import libraries and download data
Let's start with importing the necessary libraries. 

In [ ]:
!pip install neuralnets

In [ ]:
import pickle
import numpy as np
import random
import os
import matplotlib.pyplot as plt
plt.rcdefaults()
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from progressbar import ProgressBar, Percentage, Bar, ETA, FileTransferSpeed
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import gdown
import zipfile
import os
import progressbar
import time

from neuralnets.util.io import read_tif

As you will notice, Colab environments come with quite a large library pre-installed. If you need to import a module that is not yet specified, you can add it in the previous cell (make sure to run it again). If the module is not installed, you can install it with `pip`. 

To make your work reproducible, it is advised to initialize all modules with stochastic functionality with a fixed seed. Re-running this script should give the same results as long as the seed is fixed. 

In [ ]:
# make sure the results are reproducible
seed = 0
np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# run all computations on the GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Running computations with %s' % torch.device(device))
if torch.cuda.is_available():
    print(torch.cuda.get_device_properties(device))

We will now download the required data from a public Google Drive repository. The data is stored as a zip archive and automatically extracted to the `data` directory in the current directory. 

In [ ]:
# fields
url = 'http://data.bits.vib.be/pub/trainingen/DeepLearning/data-3.zip'
cmp_data_path = 'data.zip'

# download the compressed data
gdown.download(url, cmp_data_path, quiet=False)

# extract the data
zip = zipfile.ZipFile(cmp_data_path)
zip.extractall('')

# remove the compressed data
os.remove(cmp_data_path)

## 2. Designing a generative adversarial network

To get started with generative adversarial networks, we will design a model for the MNIST dataset. The following code will download the data (if necessary) and initialize a data loader. 

In [ ]:
# parameters
batch_size = 32

# transform object that pre-processes the data
transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=0.5, std=0.5)
])

# data loader object
loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=True, download=True, transform=transform), 
    batch_size=batch_size, shuffle=True)

# show a few samples
n = 5
for i in range(n): 
    plt.subplot(1, n, i+1)
    x = loader.dataset.data[i]
    plt.imshow(x.numpy(), cmap='gray')
    plt.axis('off')
plt.show()

Note that we only load the training data, as testing is not really necessary for generative models. The objective of a generative model is to generate good samples, not to maximize clustering or classification performance. To define an objective function for a generative model, remains an open problem up to now. 

Also denote that we apply a transformation that converts the data to tensors and normalizes the data to the $[-1,1]$ range. We will exploit this in our network architecture to simplify data generation. 

We can now start building the generative adversarial network. As seen in the theory, the architecture consists of a generator and discriminator. Both networks consist of 4 linear layers with [leaky ReLU](https://pytorch.org/docs/stable/generated/torch.nn.LeakyReLU.html) activation (except for the output activations). The generator transforms a latent noise vector into an vector represented image. The discriminator takes an image as input and computes the likelihood that this sample is real. Note that the output activation of the generator is a [tanh](https://pytorch.org/docs/stable/generated/torch.tanh.html) function (conform with the inputs that have been scaled to $[-1,1]$), and the output activation of the discriminator is a [sigmoid](https://pytorch.org/docs/stable/generated/torch.sigmoid.html) function to correspond with a probability. The image below shows the complete GAN architecture (top: generator, bottom: discriminator) in full detail. 

![GAN scheme](https://i.ibb.co/v15VJph/scheme2.png)

**Exercise**: Implement the above GAN architecture: 
- Implement the initialization and forward functions of the Generator and Discriminator classes. 
- Print out the architecture

In [ ]:
class Generator(nn.Module):

    def __init__(self, input_size, output_size):
        super(Generator, self).__init__()

        """
        INSERT CODE HERE
        """

    def forward(self, x):

        """
        INSERT CODE HERE
        """

        return x

class Discriminator(nn.Module):
    
    def __init__(self, input_size, output_size):
        super(Discriminator, self).__init__()
        
        """
        INSERT CODE HERE
        """

    def forward(self, x):
        
        """
        INSERT CODE HERE
        """

        return x
        
"""
INSERT CODE HERE
"""

## 3. Training a generative adversarial network

Now comes the tricky part: training a GAN from scratch. 

**Exercise**: Train the GAN network on the MNIST dataset. The code below has most components filled to train a GAN network. Take your time and go through it. In each epoch, the discriminator is first trained w.r.t. its loss function. Next, the generator is updated w.r.t. its loss function. At the end of each epoch, the generator computes its generated images from a fixed noise vector and you will see the generated results. Keep in mind that an epoch might require a few seconds to finish due to the many forward and backward passes. 
- Complete the `train_discriminator` function by implementing what is mentioned in the comments. 
- Complete the `train_generator` function by implementing what is mentioned in the comments. 
- Analyse how the generated samples evolve during training
- Study the training curves and explain why the discriminator/generator loss increases/decreases. Note that we plot half of the discriminator loss because this is originally the sum of the losses for a real and fake sample. 

In [ ]:
# show the generated samples that correspond to a (5x5) batch of noise vectors
def show_generated_samples(G, z):

    # set generator to evaluation mode and run it
    G.eval()
    test_images = G(z)

    # figure stuff
    size_figure_grid = 5
    fig, ax = plt.subplots(size_figure_grid, size_figure_grid, figsize=(5, 5))
    for i in range(size_figure_grid):
        for j in range(size_figure_grid):
            ax[i, j].get_xaxis().set_visible(False)
            ax[i, j].get_yaxis().set_visible(False)

    # visualize the generated images
    for k in range(5*5):
        i = k // 5
        j = k % 5
        ax[i, j].cla()
        ax[i, j].imshow(test_images[k, :].cpu().detach().view(28, 28).numpy(), cmap='gray')
    plt.show()

# train the discriminator for a single iteration
def train_discriminator(D, G, x, D_optimizer): 

    # zero the gradients
    D.zero_grad()

    # domain labels: real corresponds to 1, fake corresponds to 0
    y_real = torch.ones(x.size(0)).to(device)
    y_fake = torch.zeros(x.size(0)).to(device)

    # run the discriminator on the real sample and compute the loss
    """
    INSERT CODE HERE
    """

    # generate a fake sample
    """
    INSERT CODE HERE
    """

    # run the discriminator on the fake sample and compute the loss
    """
    INSERT CODE HERE
    """

    # sum the real and fake loss
    D_train_loss = D_real_loss + D_fake_loss

    # backpropagation on the discriminator and parameter update
    D_train_loss.backward()
    D_optimizer.step()

    discriminator_loss = D_train_loss.item()

    return discriminator_loss

# train the generator for a single iteration
def train_generator(D, G, G_optimizer): 

    # zero the gradients
    G.zero_grad()
    
    # domain labels: real corresponds to 1, fake corresponds to 0
    y = torch.ones(batch_size).to(device)


    # generate a fake sample
    """
    INSERT CODE HERE
    """

    # run the discriminator on the fake sample and compute the loss
    """
    INSERT CODE HERE
    """

    # backpropagation on the generator and parameter update
    G_fake_loss.backward()
    G_optimizer.step()

    generator_loss = G_fake_loss.item()

    return generator_loss

# train the GAN network for a number of epochs
def train_gan(D, G, loader, D_optimizer, G_optimizer, epochs, fixed_z):

    loss_d = np.zeros((epochs))
    loss_g = np.zeros((epochs))
    for epoch in range(epochs):

        # set generator and discriminator to training mode
        G.train()
        D.train()

        for x, _ in loader:
            
            # get a batch of samples
            x = x.view(-1, 28 * 28).to(device).float()

            # train discriminator D and cumulate loss
            loss = train_discriminator(D, G, x, D_optimizer)
            loss_d[epoch] += loss

            # train generator G and cumulate loss
            loss = train_generator(D, G, G_optimizer)
            loss_g[epoch] += loss

        # average losses and report
        loss_d[epoch] /= len(loader)
        loss_g[epoch] /= len(loader)
        print('Epoch %d: Discriminator loss = %.3f - Generator loss = %.3f' % (epoch, loss_d[epoch], loss_g[epoch]))

        # show generated samples
        show_generated_samples(G, fixed_z)

    return loss_d, loss_g


# training parameters
lr = 2e-4
epochs = 10

# binary cross entropy loss is used in a GAN
BCE_loss = nn.BCELoss()

# adam optimizer for both networks
G_optimizer = optim.Adam(G.parameters(), lr=lr)
D_optimizer = optim.Adam(D.parameters(), lr=lr)

# fixed noise (for consistency in comparing generated samples)
fixed_z = torch.randn((5 * 5, G.input_size)).to(device)

# train the network
loss_d, loss_g = train_gan(D, G, loader, D_optimizer, G_optimizer, epochs, fixed_z)

# visualize the learning curves
plt.plot(loss_d / 2)
plt.plot(loss_g)
plt.legend(('Discriminator', 'Generator'))
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.show()

Keep in mind that training GAN networks can be tedious and time-consuming. Hyperparameter settings and architectural choices such as where to include dropout and normalization are crucial. Feel free to try this out by e.g. removing the dropout layers, changing the learning rate, etc.  

## 4. Adaptation to electron microscopy data

We will now adapt the GAN for digit image generation to a generative model for electron microscopy data. Specifically, the data used originates from the ISBI 2012 segmentation challenge on neuron structures (see third exercise session). As our model is unsupervised, we will not used the labels provided in the dataset. We briefly visualize a slice of the data. 

In [ ]:
# specify where the data is stored
data_dir = 'data-3'

# load the datadispensers
x = read_tif(os.path.join(data_dir, 'train-volume.tif'))

# print out size
print('Size of the volume: %d x %d x %d' % x.shape)

# show example
plt.imshow(x[0], cmap='gray')
plt.title('Input data')
plt.axis('off')
plt.show()

We will try to re-use the code from the MNIST dataset as much as possible. To make our lives easier, we will therefore attempt to generate EM samples of size $28 \times 28$ as well. To do this, we need a data loader that samples random windows of this size from the data volume. 

**Exercise**: Implement a data loader that returns windows of size $28 \times 28$. 
- Implement the `__init__`, `__getitem__` and `__len__` function of the `EMWindowDataset` class. The initializer should save variables that are necessary for sampling (e.g. the window size). The length of the dataset can simply be a fixed number of iterations (e.g. $10^4$). For sampling, you should pick a random location in the volume and return a window of size $28 \times 28$. 
- Make sure that your sampled window is rescaled to the $[-1, 1]$ interval, as with the MNIST data. 
- The additional code below shows the result of your sampling operation. 

In [ ]:
# dataset useful for sampling (and many other things)
class EMWindowDataset(data.Dataset):
    
    def __init__(self, x, wnd_sz, max_iter_epoch):

        """
        INSERT CODE HERE
        """
    
    def __getitem__(self, i):

        """
        INSERT CODE HERE
        """

        return wnd
    
    def __len__(self):
        """
        INSERT CODE HERE
        """

"""
INSERT CODE HERE
"""

Now we are reade to train the GAN on EM data! 

**Exercise**: Below is the `train_gan` function of the MNIST dataset. The only thing you will have to adjust is the data loading step. Keep in mind that the MNIST dataset returns a tuple (the input and a label), whereas our data loader returns only the necessary image. To train the network, we recommend to reduce the learning rate to $10^{-5}$ and increase the amount of epochs to $20$. 

In [ ]:
# train the GAN network for a number of epochs
def train_gan(D, G, loader, D_optimizer, G_optimizer, epochs, fixed_z):

    loss_d = np.zeros((epochs))
    loss_g = np.zeros((epochs))
    for epoch in range(epochs):

        # set generator and discriminator to training mode
        G.train()
        D.train()

        for x, _ in loader:
            
            # get a batch of samples
            x = x.view(-1, 28 * 28).to(device).float()

            # train discriminator D and cumulate loss
            loss = train_discriminator(D, G, x, D_optimizer)
            loss_d[epoch] += loss

            # train generator G and cumulate loss
            loss = train_generator(D, G, G_optimizer)
            loss_g[epoch] += loss

        # average losses and report
        loss_d[epoch] /= len(loader)
        loss_g[epoch] /= len(loader)
        print('Epoch %d: Discriminator loss = %.3f - Generator loss = %.3f' % (epoch, loss_d[epoch], loss_g[epoch]))

        # show generated samples
        show_generated_samples(G, fixed_z)

    return loss_d, loss_g


# parameters
z_dim = 64
wnd_sz = 28
lr = 2e-4
epochs = 10

# networks
G = Generator(input_size=z_dim, output_size=wnd_sz**2).to(device)
D = Discriminator(input_size=wnd_sz**2, output_size=1).to(device)

# binary cross entropy loss is used in a GAN
BCE_loss = nn.BCELoss()

# adam optimizer for both networks
G_optimizer = optim.Adam(G.parameters(), lr=lr)
D_optimizer = optim.Adam(D.parameters(), lr=lr)

# fixed noise (for consistency in comparing generated samples)
fixed_z = torch.randn((5 * 5, G.input_size)).to(device)

# train the network
loss_d, loss_g = train_gan(D, G, loader, D_optimizer, G_optimizer, epochs, fixed_z)

# visualize the learning curves
plt.plot(loss_d / 2)
plt.plot(loss_g)
plt.legend(('Discriminator', 'Generator'))
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.show()

Lastly, these models allow you to interpolate in a very realistic sense between generated samples. Consider two random noise vectors $\mathbf{z}$ and $\mathbf{z}'$, and their corresponding generated samples, $\mathbf{x}=G(\mathbf{z})$ and $\mathbf{x}'=G(\mathbf{z}')$, respectively. An interpolated sample between the two is then obtained by interpolating in the latent space and decoding with the generator, i.e. $\mathbf{x}_\alpha=G(\alpha \mathbf{z} + (1 - \alpha) \mathbf{z}')$. The code below shows this visually. 

In [ ]:
# generate z and z'
z = torch.randn((1, G.input_size)).to(device)
z_ = torch.randn((1, G.input_size)).to(device)

# generate interpolations between z and z'
n = 12
zi = torch.zeros((n, G.input_size)).to(device)
for i in range(1, n+1): 
    alpha = i / (n+1)
    zi[i-1] = alpha * z[0] + (1 - alpha) * z_[0]

# concatenate all samples in a single batch
z_all = torch.cat((z, zi, z_), dim=0)

# decode the samples with the generator
x_all = G(z_all)

# visualize samples
for i in range(n+1): 
    plt.subplot(1, n+1, i+1)
    plt.imshow(x_all[i].detach().cpu().view(28, 28).numpy(), cmap='gray')
    plt.axis('off')
plt.show()